In [1]:
import os
import pandas as pd
import warnings
import matplotlib.pyplot as plt

import numpy as np
import seaborn as sns
import sys

from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm
import time


plt.rcParams["figure.figsize"] = (9, 6)
warnings.simplefilter("ignore")

sys.path.append("../1_Format_donnees/")
from aggregation import load_and_preprocess_agg_metrics, Scaler_Metrics
sys.path.append("../")

from evaluate import plot_confusion_matrix, get_scores, plot_feature_importance

### <8 mbits en cumsum = bruit 

In [2]:
WINDOW_SIZE = 60
STEP = 15
MIN_DURA = 15

In [3]:
time_counter = time.time_ns()

df_test = load_and_preprocess_agg_metrics("../Mesures/test", window_size = WINDOW_SIZE, step = STEP, min_duration = MIN_DURA )
df_train = load_and_preprocess_agg_metrics("../Mesures/train", window_size = WINDOW_SIZE, step = STEP)

process_duration = (time.time_ns() - time_counter)/(10**9)
print(f"Dataset generation took : {process_duration} s")

Load datasets...
Extracting ../Mesures/test\messenger20221124_151137.csv.... 1/7


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 15.89it/s]


Extracting ../Mesures/test\messenger20230208_132747.csv.... 2/7


100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:05<00:00,  4.67it/s]


Extracting ../Mesures/test\skype20221206_150902.csv.... 3/7


...

Splitting the data in windows...


100%|██████████████████████████████████████████████████████████████████████████████| 5593/5593 [02:46<00:00, 33.58it/s]


Computing metrics and statistics...


 27%|████████████████████▍                                                        | 6297/23657 [03:42<10:14, 28.27it/s]


KeyboardInterrupt: 

In [ ]:
scaler = Scaler_Metrics()
training_data = scaler.fit_transform(df_train)
testing_data = scaler.transform(df_test)
training_data

In [ ]:
training_data = training_data[training_data .label!="noise"]
testing_data = testing_data[testing_data .label!="noise"]

In [ ]:
time_counter = time.time_ns()

model = RandomForestClassifier(class_weight = "balanced")
model.fit(training_data[training_data.columns[:-1]], training_data.label)

training_duration = (time.time_ns() - time_counter)/(10**9)
print(f"Training Duration : {training_duration} s for a dataset length of {len(training_data.label)}")

In [ ]:
# mesure efficiency
time_counter = time.time_ns()

preds_labels =  model.predict(testing_data[testing_data.columns[:-1]])
true_labels = testing_data.label

testing_duration = (time.time_ns() - time_counter)/(10**9)
print(f"Testing Duration : {testing_duration} s for a dataset length of {len(testing_data.label)}")
testing_duration_per_item = testing_duration*(10**3)/len(testing_data.label)
print(f"Testing Duration per data : {testing_duration_per_item} ms")

In [ ]:
plot_confusion_matrix(true_labels,preds_labels)

In [ ]:
get_scores(true_labels,preds_labels)

In [ ]:

    
plot_feature_importance(model, training_data.columns[:-1])